In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.preprocessing import StandardScaler


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-5-0/train_kaggle.csv
/kaggle/input/data-storm-5-0/test_kaggle.csv


In [2]:
train_df = pd.read_csv("/kaggle/input/data-storm-5-0/train_kaggle.csv")
test_df = pd.read_csv("/kaggle/input/data-storm-5-0/test_kaggle.csv")
test_df.head()

/tmp/ipykernel_18/2988013644.py:1: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("/kaggle/input/data-storm-5-0/train_kaggle.csv")


,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
0,33574,batticaloa,2686.5,3582,12537
1,10089,batticaloa,1717.56,2576.34,9446.58
2,38329,batticaloa,854.04,1242.24,5201.88
3,11376,batticaloa,1638.12,2320.67,9282.68
4,12410,batticaloa,1039.09,1518.67,5435.24


In [3]:
train_df = pd.read_csv("/kaggle/input/data-storm-5-0/train_kaggle.csv")


/tmp/ipykernel_18/398655442.py:1: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("/kaggle/input/data-storm-5-0/train_kaggle.csv")


In [4]:
test_df = pd.read_csv("/kaggle/input/data-storm-5-0/test_kaggle.csv")
test_df.head()

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
0,33574,batticaloa,2686.5,3582,12537
1,10089,batticaloa,1717.56,2576.34,9446.58
2,38329,batticaloa,854.04,1242.24,5201.88
3,11376,batticaloa,1638.12,2320.67,9282.68
4,12410,batticaloa,1039.09,1518.67,5435.24


# Convert dry_sales

In [5]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
train_df['dry_sales_numeric'] = pd.to_numeric(train_df['dry_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = train_df[train_df['dry_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'dry_sales']])


        Customer_ID                              dry_sales
4929     10414692.0                                    NaN
78400    10504943.0                                    NaN
148029   10190107.0  Four thousand and two hundread ruppes
174631   10273730.0                                    nul
174720   10039004.0                                    nul
175278   10347733.0                                    nul
268571   10047014.0                                    nul
296621   10215937.0                                    nul
327168   10649653.0                                    NaN
327169   10737280.0                                    NaN
327170   10010850.0                                    NaN
327171   10347930.0                                    NaN
327172   10351890.0                                    NaN
327173   10025449.0                                    NaN
327174   10638161.0                                    NaN
327495   10099173.0                                    n

In [6]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10190107, 'dry_sales'] = 4200

# Verify the change
print(train_df[train_df['Customer_ID'] == 10190107])


        Customer_ID             outlet_city luxury_sales fresh_sales  \
148029   10190107.0  Dehiwala-Mount Lavinia        931.5       931.5   

       dry_sales cluster_catgeory  dry_sales_numeric  
148029      4200                4                NaN  


# Convert fresh_sales

In [7]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
train_df['fresh_sales_numeric'] = pd.to_numeric(train_df['fresh_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = train_df[train_df['fresh_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'fresh_sales']])


        Customer_ID                          fresh_sales
4326     10531406.0                                  NaN
4327     10311229.0                                  NaN
4333     10361798.0                                  NaN
4929     10414692.0                                  NaN
5379     10600529.0                                  NaN
5403     10484539.0                                  NaN
5408     10321643.0                                  NaN
78367    10424117.0                                  NaN
78380    10343473.0                                  NaN
78400    10504943.0                                  NaN
161810   10261075.0         Six hundread and five ruppes
174714   10358037.0                                  nul
268571   10047014.0                                  nul
326334   10006007.0     Three thousana and five hundread
326763   10301959.0                                  NaN
326764   10671014.0                                  NaN
326765   10687830.0            

In [8]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10006007.0 , 'fresh_sales'] = 3500

# Verify the change
print(train_df[train_df['Customer_ID'] == 10006007.0 ])


        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
326334   10006007.0    Kaduwela       758.52        3500   1029.42   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  
326334                2            1029.42                  NaN  


In [9]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10314616.0  , 'fresh_sales'] = 13000

# Verify the change
print(train_df[train_df['Customer_ID'] == 10314616.0  ])


        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
558897   10314616.0  Batticaloa      3027.36       13000   2081.31   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  
558897              6.0            2081.31                  NaN  


In [10]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10369123.0  , 'fresh_sales'] = 5000

# Verify the change
print(train_df[train_df['Customer_ID'] == 10369123.0  ])


        Customer_ID  outlet_city luxury_sales fresh_sales dry_sales  \
752816   10369123.0  Trincomalee      5501.57        5000   5679.04   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  
752816                3            5679.04                  NaN  


In [11]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10261075.0 , 'fresh_sales'] = 605

# Verify the change
print(train_df[train_df['Customer_ID'] == 10261075.0 ])

        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
161810   10261075.0    Panadura       887.36         605    3604.9   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  
161810                4             3604.9                  NaN  


In [12]:
train_df.loc[train_df['Customer_ID'] == 10369199, 'fresh_sales'] = 2700

# Verify the change
print(train_df[train_df['Customer_ID'] == 10369199])

        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
753256   10369199.0     Colombo       2692.8        2700    2861.1   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  
753256                1             2861.1                  NaN  


In [13]:
train_df.loc[train_df['Customer_ID'] == 10369199, 'fresh_sales'] = 2700

# Verify the change
print(train_df[train_df['Customer_ID'] == 10369199])

        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
753256   10369199.0     Colombo       2692.8        2700    2861.1   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  
753256                1             2861.1                  NaN  


# Convert luxury_values

In [14]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
train_df['luxury_sales_numeric'] = pd.to_numeric(train_df['luxury_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = train_df[train_df['luxury_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'luxury_sales']])


        Customer_ID                         luxury_sales
4234     10101878.0                                  NaN
4333     10361798.0                                  NaN
5375     10556752.0                                  NaN
5408     10321643.0                                  NaN
78367    10424117.0                                  NaN
78380    10343473.0                                  NaN
80043    10774052.0    One thousand four hundread ruppes
174628   10207785.0                                  NaN
175278   10347733.0                                  nul
296621   10215937.0                                  nul
297911   10189925.0               Eight hundread ruppess
326593   10467590.0              six hundread and hirty 
327258   10615996.0                                  NaN
327259   10346205.0                                  NaN
327260   10722463.0                                  NaN
327261   10221069.0                                  NaN
329847   10328520.0            

In [15]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10774052.0 , 'luxury_sales'] = 1400

# Verify the change
print(train_df[train_df['Customer_ID'] == 10774052.0 ])

       Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
80043   10774052.0    Kelaniya         1400      1118.0    5934.0   

      cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  \
80043                4             5934.0               1118.0   

       luxury_sales_numeric  
80043                   NaN  


In [16]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10189925.0 , 'luxury_sales'] = 800

# Verify the change
print(train_df[train_df['Customer_ID'] == 10189925.0])

        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
297911   10189925.0    Kelaniya          800     4003.84   1059.84   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  \
297911                2            1059.84              4003.84   

        luxury_sales_numeric  
297911                   NaN  


In [17]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10467590.0  , 'luxury_sales'] = 630

# Verify the change
print(train_df[train_df['Customer_ID'] == 10467590.0 ])

        Customer_ID   outlet_city luxury_sales fresh_sales dry_sales  \
326593   10467590.0  Nuwara Eliya          630      3420.3    894.54   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  \
326593                2             894.54               3420.3   

        luxury_sales_numeric  
326593                   NaN  


In [18]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10537755.0   , 'luxury_sales'] = 1200

# Verify the change
print(train_df[train_df['Customer_ID'] == 10537755.0  ])

        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
497177   10537755.0       Kandy         1200      2041.0    6939.4   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  \
497177                1             6939.4               2041.0   

        luxury_sales_numeric  
497177                   NaN  


In [19]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10468127.0   , 'luxury_sales'] = 709

# Verify the change
print(train_df[train_df['Customer_ID'] == 10468127.0  ])

        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
497245   10468127.0       Kandy          709     1032.48   4194.45   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  \
497245                1            4194.45              1032.48   

        luxury_sales_numeric  
497245                   NaN  


In [20]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] ==  10076064.0  , 'luxury_sales'] = 3200

# Verify the change
print(train_df[train_df['Customer_ID'] ==  10076064.0])

        Customer_ID  outlet_city luxury_sales fresh_sales dry_sales  \
558562   10076064.0  Trincomalee         3200     10416.9   1923.12   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  \
558562              6.0            1923.12              10416.9   

        luxury_sales_numeric  
558562                   NaN  


In [21]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10695296.0   , 'luxury_sales'] = 4100

# Verify the change
print(train_df[train_df['Customer_ID'] == 10695296.0  ])

        Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
753131   10695296.0  Batticaloa         4100     4286.72   4034.56   

       cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  \
753131                3            4034.56              4286.72   

        luxury_sales_numeric  
753131                   NaN  


# ClusterCategory

In [22]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
train_df['clusterCategory_numeric'] = pd.to_numeric(train_df['cluster_catgeory'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = train_df[train_df['clusterCategory_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'cluster_catgeory']])


        Customer_ID cluster_catgeory
3560     10604532.0               6\
555560   10158111.0              NaN


In [23]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
train_df.loc[train_df['Customer_ID'] == 10604532, 'cluster_catgeory'] = 6

# Verify the change
print(train_df[train_df['Customer_ID'] == 10604532])

      Customer_ID             outlet_city luxury_sales fresh_sales dry_sales  \
3560   10604532.0  Dehiwala-Mount Lavinia      1811.34     1408.82   6641.58   

     cluster_catgeory  dry_sales_numeric  fresh_sales_numeric  \
3560                6            6641.58              1408.82   

      luxury_sales_numeric  clusterCategory_numeric  
3560               1811.34                      NaN  


In [24]:

# Remove the row with Customer_ID 10158111.0
train_df = train_df[train_df['Customer_ID'] != 10158111.0]

# Verify the removal
print(train_df[train_df['Customer_ID'] == 10158111.0])


Empty DataFrame
Columns: [Customer_ID, outlet_city, luxury_sales, fresh_sales, dry_sales, cluster_catgeory, dry_sales_numeric, fresh_sales_numeric, luxury_sales_numeric, clusterCategory_numeric]
Index: []


In [25]:
# Find the mode of 'cluster_category'
mode_cluster_category = train_df['cluster_catgeory'].mode()

print("Mode of cluster_category:", mode_cluster_category)

Mode of cluster_category: 0    1
Name: cluster_catgeory, dtype: object


In [26]:
# Convert 'cluster_catgeory' column to numeric data type
train_df['cluster_catgeory'] = pd.to_numeric(train_df['cluster_catgeory'], errors='coerce')



In [27]:
# Count rows before removal
rows_before_removal = len(train_df)

# Remove rows where cluster_catgeory is greater than 6
train_df = train_df[train_df['cluster_catgeory'] <= 6]

# Count rows after removal
rows_after_removal = len(train_df)

# Calculate the number of rows removed
rows_removed = rows_before_removal - rows_after_removal

print("Number of rows removed:", rows_removed)


Number of rows removed: 5


In [28]:
train_df.describe()

,Customer_ID,cluster_catgeory,dry_sales_numeric,fresh_sales_numeric,luxury_sales_numeric,clusterCategory_numeric
count,7.741470e+05,774149.000000,774111.000000,774099.000000,774104.000000,774148.000000
mean,1.038708e+07,3.292104,4676.283901,4428.685403,1921.970078,3.292100
std,2.234797e+05,1.867014,3409.386412,3334.660285,1004.080045,1.867013
min,1.000000e+07,1.000000,500.000000,500.000000,500.000000,1.000000
25%,1.019354e+07,2.000000,1787.940000,1620.190000,1213.530000,2.000000
50%,1.038708e+07,3.000000,3727.760000,3356.480000,1715.280000,3.000000
75%,1.058062e+07,5.000000,7162.200000,6671.610000,2338.700000,5.000000
max,1.077415e+07,6.000000,13999.300000,13997.900000,6999.650000,6.000000


mode is 1

In [29]:
unique_cluster_categories = train_df['cluster_catgeory'].unique()
print(unique_cluster_categories)


[4. 1. 2. 5. 3. 6.]


In [30]:
train_df['cluster_catgeory'] = train_df['cluster_catgeory'].astype(int)


# OUTLET city in train_df and test_df check

* train_data

In [31]:
unique_city_categories = train_df['outlet_city'].unique()
print(unique_city_categories)

['Kelaniya' 'Moratuwa' 'Wattala' 'Homagama' 'Dehiwala-Mount Lavinia'
 'Panadura' 'Kaduwela' 'Peliyagoda' 'Kotte' 'Nuwara Eliya' 'Batticaloa'
 'Colombo' 'Jaffna' 'Gampaha' 'Kalmunai' 'Galle' 'Katunayake' 'Negombo'
 'Trincomalee' 'Kandy' nan]


In [32]:
# Find the mode of 'outlet_city'
mode_outlet_city = train_df['outlet_city'].mode()[0]

# Replace NaN values with the mode value
train_df['outlet_city'] = train_df['outlet_city'].fillna(mode_outlet_city)


In [33]:


# One-hot encode the 'outlet_city' column
train_df = pd.get_dummies(train_df, columns=['outlet_city'])

# Display the first few rows of the DataFrame to verify the changes
print(train_df.head())


   Customer_ID luxury_sales fresh_sales dry_sales  cluster_catgeory  \
0   10493832.0       1209.6       756.0    5292.0                 4   
1   10178643.0      1590.12     1060.08   6007.12                 1   
2   10513916.0      2288.88     1481.04   9155.52                 4   
3   10334589.0      2274.94     1739.66   9099.76                 4   
4   10458365.0      2345.49     2069.55   9243.99                 4   

   dry_sales_numeric  fresh_sales_numeric  luxury_sales_numeric  \
0            5292.00               756.00               1209.60   
1            6007.12              1060.08               1590.12   
2            9155.52              1481.04               2288.88   
3            9099.76              1739.66               2274.94   
4            9243.99              2069.55               2345.49   

   clusterCategory_numeric  outlet_city_Batticaloa  ...  \
0                      4.0                   False  ...   
1                      1.0                   False  

In [34]:
train_df.columns

Index(['Customer_ID', 'luxury_sales', 'fresh_sales', 'dry_sales',
       'cluster_catgeory', 'dry_sales_numeric', 'fresh_sales_numeric',
       'luxury_sales_numeric', 'clusterCategory_numeric',
       'outlet_city_Batticaloa', 'outlet_city_Colombo',
       'outlet_city_Dehiwala-Mount Lavinia', 'outlet_city_Galle',
       'outlet_city_Gampaha', 'outlet_city_Homagama', 'outlet_city_Jaffna',
       'outlet_city_Kaduwela', 'outlet_city_Kalmunai', 'outlet_city_Kandy',
       'outlet_city_Katunayake', 'outlet_city_Kelaniya', 'outlet_city_Kotte',
       'outlet_city_Moratuwa', 'outlet_city_Negombo',
       'outlet_city_Nuwara Eliya', 'outlet_city_Panadura',
       'outlet_city_Peliyagoda', 'outlet_city_Trincomalee',
       'outlet_city_Wattala'],
      dtype='object')

* test_df

In [35]:
unique_city_categories = test_df['outlet_city'].unique()
print(unique_city_categories)

['batticaloa' 'Batticaloa' 'Colombo' 'Dehiwala-Mount Lavinia'
 'Anuradhapura' 'Galle' 'Gampaha' 'Homagama' 'Jaffna' 'Kaduwela'
 'Kalmunai' 'kalmunai' 'Kandy' 'Katunayake' 'Kelaniya' 'Madawachiya'
 'Kotte' 'Moratuwa' 'MoraTuwa' 'Negombo' 'Nuwara Eliya' 'Panadura'
 'Peliyagoda' 'PeliyagodA' 'Trincomale' 'Trincomalee' 'Wattala']


# Check unique values 

1. luxury_sales

In [36]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
train_df['luxury_sales_numeric'] = pd.to_numeric(train_df['luxury_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = train_df[train_df['luxury_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'luxury_sales']])


        Customer_ID luxury_sales
4234     10101878.0          NaN
4333     10361798.0          NaN
5375     10556752.0          NaN
5408     10321643.0          NaN
78367    10424117.0          NaN
78380    10343473.0          NaN
174628   10207785.0          NaN
175278   10347733.0          nul
296621   10215937.0          nul
327258   10615996.0          NaN
327259   10346205.0          NaN
327260   10722463.0          NaN
327261   10221069.0          NaN
329847   10328520.0          NaN
329848   10155003.0          NaN
329849   10600486.0          NaN
352298   10703170.0          NaN
352662   10638607.0          NaN
352663   10223839.0          NaN
352664   10418282.0          NaN
367935   10445384.0          nul
496140   10616379.0          NaN
497485   10751852.0          NaN
554998   10420777.0          NaN
555305   10224685.0          NaN
555469   10692500.0          NaN
555573   10294701.0          NaN
555610   10262970.0          NaN
555764   10335139.0          NaN
722325   1

2. fresh_Sales

In [37]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
train_df['fresh_sales_numeric'] = pd.to_numeric(train_df['fresh_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = train_df[train_df['fresh_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'fresh_sales']])


        Customer_ID fresh_sales
4326     10531406.0         NaN
4327     10311229.0         NaN
4333     10361798.0         NaN
4929     10414692.0         NaN
5379     10600529.0         NaN
5403     10484539.0         NaN
5408     10321643.0         NaN
78367    10424117.0         NaN
78380    10343473.0         NaN
78400    10504943.0         NaN
174714   10358037.0         nul
268571   10047014.0         nul
326763   10301959.0         NaN
326764   10671014.0         NaN
326765   10687830.0         NaN
327495   10099173.0         nul
327505   10312003.0         nul
329931   10540582.0         NaN
352298   10703170.0         NaN
352658   10418309.0         NaN
354153   10210509.0         NaN
496138   10559183.0         NaN
497003   10367366.0         NaN
497445   10168166.0         NaN
497446   10545713.0         NaN
497447   10624363.0         NaN
497598   10570526.0         NaN
555470   10403312.0         NaN
555607   10488405.0         NaN
555764   10335139.0         NaN
628150  

3.dry_sales

In [38]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
train_df['dry_sales_numeric'] = pd.to_numeric(train_df['dry_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = train_df[train_df['dry_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'dry_sales']])


        Customer_ID dry_sales
4929     10414692.0       NaN
78400    10504943.0       NaN
174631   10273730.0       nul
174720   10039004.0       nul
175278   10347733.0       nul
268571   10047014.0       nul
296621   10215937.0       nul
327168   10649653.0       NaN
327169   10737280.0       NaN
327170   10010850.0       NaN
327171   10347930.0       NaN
327172   10351890.0       NaN
327173   10025449.0       NaN
327174   10638161.0       NaN
327495   10099173.0       nul
327505   10312003.0       nul
329931   10540582.0       NaN
345090   10186598.0       NaN
345091   10215208.0       NaN
345092   10530946.0       NaN
353317   10659375.0       NaN
496135   10501123.0       NaN
497003   10367366.0       NaN
497464   10155270.0       NaN
497583   10234709.0       NaN
497597   10345956.0       NaN
554998   10420777.0       NaN
555305   10224685.0       NaN
555469   10692500.0       NaN
555536   10092366.0       NaN
555604   10063564.0       NaN
753448   10460776.0       NaN
753449   1

# 

# remove null and NAN of dry_sales,fresh_sales and luxury_Sales and fill them with mean values

In [39]:
# Convert columns 3, 4, and 5 to numeric types, coercing errors to NaN
train_df[['luxury_sales', 'fresh_sales', 'dry_sales']] = train_df[['luxury_sales', 'fresh_sales', 'dry_sales']].apply(pd.to_numeric, errors='coerce')

# Replace NaN values with mean of each column
train_df['luxury_sales'] = train_df['luxury_sales'].fillna(train_df['luxury_sales'].mean())
train_df['fresh_sales'] = train_df['fresh_sales'].fillna(train_df['fresh_sales'].mean())
train_df['dry_sales'] = train_df['dry_sales'].fillna(train_df['dry_sales'].mean())


In [40]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
train_df['Customer_ID'] = pd.to_numeric(train_df['Customer_ID'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = train_df[train_df['Customer_ID'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID']])


        Customer_ID
497678          NaN
497698          NaN


In [41]:
train_df.describe()

,Customer_ID,luxury_sales,fresh_sales,dry_sales,cluster_catgeory,dry_sales_numeric,fresh_sales_numeric,luxury_sales_numeric,clusterCategory_numeric
count,7.741470e+05,774149.000000,774149.000000,774149.000000,774149.000000,774112.000000,774104.000000,774111.000000,774148.000000
mean,1.038708e+07,1921.968250,4428.688842,4676.283285,3.292104,4676.283285,4428.688842,1921.968250,3.292100
std,2.234797e+05,1004.058305,3334.570466,3409.302777,1.867014,3409.384253,3334.667387,1004.082949,1.867013
min,1.000000e+07,500.000000,500.000000,500.000000,1.000000,500.000000,500.000000,500.000000,1.000000
25%,1.019354e+07,1213.550000,1620.220000,1788.000000,2.000000,1787.940000,1620.190000,1213.525000,2.000000
50%,1.038708e+07,1715.320000,3356.730000,3728.060000,3.000000,3727.760000,3356.510000,1715.280000,3.000000
75%,1.058062e+07,2338.690000,6671.280000,7161.760000,5.000000,7162.200000,6671.610000,2338.700000,5.000000
max,1.077415e+07,6999.650000,13997.900000,13999.300000,6.000000,13999.300000,13997.900000,6999.650000,6.000000


In [42]:
# Find the maximum existing Customer_ID
max_id = train_df['Customer_ID'].max()

# Identify the indices where Customer_ID is missing
missing_indices = train_df[train_df['Customer_ID'].isnull()].index

# Generate new unique IDs starting from max_id + 1
new_ids = range(int(max_id) + 1, int(max_id) + 1 + len(missing_indices))

# Assign the new IDs to the missing Customer_ID entries
train_df.loc[missing_indices, 'Customer_ID'] = new_ids

In [43]:
# Convert 'Customer_ID' to integer
train_df['Customer_ID'] = train_df['Customer_ID'].astype(int)

In [44]:
train_df.describe()

,Customer_ID,luxury_sales,fresh_sales,dry_sales,cluster_catgeory,dry_sales_numeric,fresh_sales_numeric,luxury_sales_numeric,clusterCategory_numeric
count,7.741490e+05,774149.000000,774149.000000,774149.000000,774149.000000,774112.000000,774104.000000,774111.000000,774148.000000
mean,1.038708e+07,1921.968250,4428.688842,4676.283285,3.292104,4676.283285,4428.688842,1921.968250,3.292100
std,2.234803e+05,1004.058305,3334.570466,3409.302777,1.867014,3409.384253,3334.667387,1004.082949,1.867013
min,1.000000e+07,500.000000,500.000000,500.000000,1.000000,500.000000,500.000000,500.000000,1.000000
25%,1.019354e+07,1213.550000,1620.220000,1788.000000,2.000000,1787.940000,1620.190000,1213.525000,2.000000
50%,1.038708e+07,1715.320000,3356.730000,3728.060000,3.000000,3727.760000,3356.510000,1715.280000,3.000000
75%,1.058062e+07,2338.690000,6671.280000,7161.760000,5.000000,7162.200000,6671.610000,2338.700000,5.000000
max,1.077416e+07,6999.650000,13997.900000,13999.300000,6.000000,13999.300000,13997.900000,6999.650000,6.000000


In [45]:
duplicates = train_df.duplicated()
duplicates.sum()

0

> # three sales are sorted

In [46]:
train_df.dtypes

Customer_ID                             int64
luxury_sales                          float64
fresh_sales                           float64
dry_sales                             float64
cluster_catgeory                        int64
dry_sales_numeric                     float64
fresh_sales_numeric                   float64
luxury_sales_numeric                  float64
clusterCategory_numeric               float64
outlet_city_Batticaloa                   bool
outlet_city_Colombo                      bool
outlet_city_Dehiwala-Mount Lavinia       bool
outlet_city_Galle                        bool
outlet_city_Gampaha                      bool
outlet_city_Homagama                     bool
outlet_city_Jaffna                       bool
outlet_city_Kaduwela                     bool
outlet_city_Kalmunai                     bool
outlet_city_Kandy                        bool
outlet_city_Katunayake                   bool
outlet_city_Kelaniya                     bool
outlet_city_Kotte                 

In [47]:
train_df.isnull().sum()

Customer_ID                            0
luxury_sales                           0
fresh_sales                            0
dry_sales                              0
cluster_catgeory                       0
dry_sales_numeric                     37
fresh_sales_numeric                   45
luxury_sales_numeric                  38
clusterCategory_numeric                1
outlet_city_Batticaloa                 0
outlet_city_Colombo                    0
outlet_city_Dehiwala-Mount Lavinia     0
outlet_city_Galle                      0
outlet_city_Gampaha                    0
outlet_city_Homagama                   0
outlet_city_Jaffna                     0
outlet_city_Kaduwela                   0
outlet_city_Kalmunai                   0
outlet_city_Kandy                      0
outlet_city_Katunayake                 0
outlet_city_Kelaniya                   0
outlet_city_Kotte                      0
outlet_city_Moratuwa                   0
outlet_city_Negombo                    0
outlet_city_Nuwa

In [48]:
train_df.dtypes

Customer_ID                             int64
luxury_sales                          float64
fresh_sales                           float64
dry_sales                             float64
cluster_catgeory                        int64
dry_sales_numeric                     float64
fresh_sales_numeric                   float64
luxury_sales_numeric                  float64
clusterCategory_numeric               float64
outlet_city_Batticaloa                   bool
outlet_city_Colombo                      bool
outlet_city_Dehiwala-Mount Lavinia       bool
outlet_city_Galle                        bool
outlet_city_Gampaha                      bool
outlet_city_Homagama                     bool
outlet_city_Jaffna                       bool
outlet_city_Kaduwela                     bool
outlet_city_Kalmunai                     bool
outlet_city_Kandy                        bool
outlet_city_Katunayake                   bool
outlet_city_Kelaniya                     bool
outlet_city_Kotte                 

# scaling

In [49]:
train_df.describe()

,Customer_ID,luxury_sales,fresh_sales,dry_sales,cluster_catgeory,dry_sales_numeric,fresh_sales_numeric,luxury_sales_numeric,clusterCategory_numeric
count,7.741490e+05,774149.000000,774149.000000,774149.000000,774149.000000,774112.000000,774104.000000,774111.000000,774148.000000
mean,1.038708e+07,1921.968250,4428.688842,4676.283285,3.292104,4676.283285,4428.688842,1921.968250,3.292100
std,2.234803e+05,1004.058305,3334.570466,3409.302777,1.867014,3409.384253,3334.667387,1004.082949,1.867013
min,1.000000e+07,500.000000,500.000000,500.000000,1.000000,500.000000,500.000000,500.000000,1.000000
25%,1.019354e+07,1213.550000,1620.220000,1788.000000,2.000000,1787.940000,1620.190000,1213.525000,2.000000
50%,1.038708e+07,1715.320000,3356.730000,3728.060000,3.000000,3727.760000,3356.510000,1715.280000,3.000000
75%,1.058062e+07,2338.690000,6671.280000,7161.760000,5.000000,7162.200000,6671.610000,2338.700000,5.000000
max,1.077416e+07,6999.650000,13997.900000,13999.300000,6.000000,13999.300000,13997.900000,6999.650000,6.000000


In [50]:
# Create 'total_sales' column by summing 'luxury_sales', 'fresh_sales', and 'dry_sales'
train_df['total_sales'] = train_df['luxury_sales'] + train_df['fresh_sales'] + train_df['dry_sales']

# Display the first few rows of the DataFrame to verify the new column
print(train_df.head())


   Customer_ID  luxury_sales  fresh_sales  dry_sales  cluster_catgeory  \
0     10493832       1209.60       756.00    5292.00                 4   
1     10178643       1590.12      1060.08    6007.12                 1   
2     10513916       2288.88      1481.04    9155.52                 4   
3     10334589       2274.94      1739.66    9099.76                 4   
4     10458365       2345.49      2069.55    9243.99                 4   

   dry_sales_numeric  fresh_sales_numeric  luxury_sales_numeric  \
0            5292.00               756.00               1209.60   
1            6007.12              1060.08               1590.12   
2            9155.52              1481.04               2288.88   
3            9099.76              1739.66               2274.94   
4            9243.99              2069.55               2345.49   

   clusterCategory_numeric  outlet_city_Batticaloa  ...  outlet_city_Kelaniya  \
0                      4.0                   False  ...                

In [51]:

# Extract the columns to be scaled
columns_to_scale = ['luxury_sales', 'fresh_sales', 'dry_sales','total_sales']

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data and transform the columns
train_df[columns_to_scale] = scaler.fit_transform(train_df[columns_to_scale])

# Testing data set

In [52]:
test_df.head()

,Customer_ID,outlet_city,luxury_sales,fresh_sales,dry_sales
0,33574,batticaloa,2686.5,3582,12537
1,10089,batticaloa,1717.56,2576.34,9446.58
2,38329,batticaloa,854.04,1242.24,5201.88
3,11376,batticaloa,1638.12,2320.67,9282.68
4,12410,batticaloa,1039.09,1518.67,5435.24


# luxury_sales

In [53]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
test_df['luxury_sales_numeric'] = pd.to_numeric(test_df['luxury_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = test_df[test_df['luxury_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'luxury_sales']])


      Customer_ID                       luxury_sales
6           10364   Seven Hundread and sixty Ruppees
6299         4059  Eight Hundread and fifteen ruppes


In [54]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
test_df.loc[test_df['Customer_ID'] == 10364  , 'luxury_sales'] =760
# Verify the change
print(test_df[test_df['Customer_ID'] == 10364  ])

   Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
6        10364  batticaloa          760       914.4    3352.8   

   luxury_sales_numeric  
6                   NaN  


In [55]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
test_df.loc[test_df['Customer_ID'] == 4059 , 'luxury_sales'] =815
# Verify the change
print(test_df[test_df['Customer_ID'] == 4059 ])

      Customer_ID   outlet_city luxury_sales fresh_sales dry_sales  \
6299         4059  Anuradhapura          815     1548.69    5705.7   

      luxury_sales_numeric  
6299                   NaN  


# fresh_sales

In [56]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
test_df['fresh_sales_numeric'] = pd.to_numeric(test_df['fresh_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = test_df[test_df['fresh_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'fresh_sales']])


       Customer_ID            fresh_sales
13949        28662  Nine thousand ruppess


In [57]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
test_df.loc[test_df['Customer_ID'] == 28662  , 'fresh_sales'] = 9000
# Verify the change
print(test_df[test_df['Customer_ID'] == 28662  ])


       Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
13949        28662      Jaffna      2195.84        9000   1509.64   

       luxury_sales_numeric  fresh_sales_numeric  
13949               2195.84                  NaN  


# dry_sales

In [58]:
# Convert 'dry_sales' column to numeric, coerce errors to NaN
test_df['dry_sales_numeric'] = pd.to_numeric(test_df['dry_sales'], errors='coerce')

# Find rows where 'dry_sales' could not be converted to numeric
invalid_rows = test_df[test_df['dry_sales_numeric'].isna()]

# Display rows with invalid 'dry_sales' values
print(invalid_rows[['Customer_ID', 'dry_sales']])


       Customer_ID                              dry_sales
40746        28664  Three Thousand seven hundread ruppees


In [59]:
# Locate the row with Customer_ID 10604532 and change the value of cluster_catgeory to 6
test_df.loc[test_df['Customer_ID'] == 28664  , 'dry_sales'] = 3700

# Verify the change
print(test_df[test_df['Customer_ID'] == 28664  ])


       Customer_ID outlet_city luxury_sales fresh_sales dry_sales  \
40746        28664     Wattala       3838.6      3951.5      3700   

       luxury_sales_numeric  fresh_sales_numeric  dry_sales_numeric  
40746                3838.6               3951.5                NaN  


In [60]:
test_df.isnull().sum()

Customer_ID             0
outlet_city             0
luxury_sales            0
fresh_sales             0
dry_sales               0
luxury_sales_numeric    2
fresh_sales_numeric     1
dry_sales_numeric       1
dtype: int64

# convert NAN values in test data set

In [61]:
# Convert columns 3, 4, and 5 to numeric types, coercing errors to NaN
test_df[['luxury_sales', 'fresh_sales', 'dry_sales']] = test_df[['luxury_sales', 'fresh_sales', 'dry_sales']].apply(pd.to_numeric, errors='coerce')

# Replace NaN values with mean of each column
test_df['luxury_sales'] = test_df['luxury_sales'].fillna(test_df['luxury_sales'].mean())
test_df['fresh_sales'] =test_df['fresh_sales'].fillna(test_df['fresh_sales'].mean())
test_df['dry_sales'] = test_df['dry_sales'].fillna(test_df['dry_sales'].mean())


In [62]:
# Create 'total_sales' column by summing 'luxury_sales', 'fresh_sales', and 'dry_sales'
test_df['total_sales'] = test_df['luxury_sales'] + test_df['fresh_sales'] + test_df['dry_sales']

# Display the first few rows of the DataFrame to verify the new column
print(test_df.head())


   Customer_ID outlet_city  luxury_sales  fresh_sales  dry_sales  \
0        33574  batticaloa       2686.50      3582.00   12537.00   
1        10089  batticaloa       1717.56      2576.34    9446.58   
2        38329  batticaloa        854.04      1242.24    5201.88   
3        11376  batticaloa       1638.12      2320.67    9282.68   
4        12410  batticaloa       1039.09      1518.67    5435.24   

   luxury_sales_numeric  fresh_sales_numeric  dry_sales_numeric  total_sales  
0               2686.50              3582.00           12537.00     18805.50  
1               1717.56              2576.34            9446.58     13740.48  
2                854.04              1242.24            5201.88      7298.16  
3               1638.12              2320.67            9282.68     13241.47  
4               1039.09              1518.67            5435.24      7993.00  


In [63]:
# Extract the columns to be scaled
columns_to_scale = ['luxury_sales', 'fresh_sales', 'dry_sales','total_sales']

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler to the data and transform the columns
test_df[columns_to_scale] = scaler.fit_transform(test_df[columns_to_scale])

In [64]:
# One-hot encode the 'outlet_city' column
test_df = pd.get_dummies(test_df, columns=['outlet_city'])

# Display the first few rows of the DataFrame to verify the changes
print(test_df.head())


   Customer_ID  luxury_sales  fresh_sales  dry_sales  luxury_sales_numeric  \
0        33574      0.756569    -0.257052   2.300097               2686.50   
1        10089     -0.208718    -0.557303   1.394065               1717.56   
2        38329     -1.068982    -0.955614   0.149628                854.04   
3        11376     -0.287859    -0.633636   1.346014               1638.12   
4        12410     -0.884630    -0.873082   0.218043               1039.09   

   fresh_sales_numeric  dry_sales_numeric  total_sales  \
0              3582.00           12537.00     2.020720   
1              2576.34            9446.58     0.699045   
2              1242.24            5201.88    -0.982024   
3              2320.67            9282.68     0.568833   
4              1518.67            5435.24    -0.800712   

   outlet_city_Anuradhapura  outlet_city_Batticaloa  ...  outlet_city_Negombo  \
0                     False                   False  ...                False   
1                   

In [65]:
# Convert 'Customer_ID' to integer
test_df['Customer_ID'] = test_df['Customer_ID'].astype(int)




In [66]:
test_df.dtypes

Customer_ID                             int64
luxury_sales                          float64
fresh_sales                           float64
dry_sales                             float64
luxury_sales_numeric                  float64
fresh_sales_numeric                   float64
dry_sales_numeric                     float64
total_sales                           float64
outlet_city_Anuradhapura                 bool
outlet_city_Batticaloa                   bool
outlet_city_Colombo                      bool
outlet_city_Dehiwala-Mount Lavinia       bool
outlet_city_Galle                        bool
outlet_city_Gampaha                      bool
outlet_city_Homagama                     bool
outlet_city_Jaffna                       bool
outlet_city_Kaduwela                     bool
outlet_city_Kalmunai                     bool
outlet_city_Kandy                        bool
outlet_city_Katunayake                   bool
outlet_city_Kelaniya                     bool
outlet_city_Kotte                 

In [67]:
train_df.dtypes

Customer_ID                             int64
luxury_sales                          float64
fresh_sales                           float64
dry_sales                             float64
cluster_catgeory                        int64
dry_sales_numeric                     float64
fresh_sales_numeric                   float64
luxury_sales_numeric                  float64
clusterCategory_numeric               float64
outlet_city_Batticaloa                   bool
outlet_city_Colombo                      bool
outlet_city_Dehiwala-Mount Lavinia       bool
outlet_city_Galle                        bool
outlet_city_Gampaha                      bool
outlet_city_Homagama                     bool
outlet_city_Jaffna                       bool
outlet_city_Kaduwela                     bool
outlet_city_Kalmunai                     bool
outlet_city_Kandy                        bool
outlet_city_Katunayake                   bool
outlet_city_Kelaniya                     bool
outlet_city_Kotte                 

# finishing cleaning the training dataset

In [68]:
train_df.describe()

,Customer_ID,luxury_sales,fresh_sales,dry_sales,cluster_catgeory,dry_sales_numeric,fresh_sales_numeric,luxury_sales_numeric,clusterCategory_numeric,total_sales
count,7.741490e+05,7.741490e+05,7.741490e+05,7.741490e+05,774149.000000,774112.000000,774104.000000,774111.000000,774148.000000,7.741490e+05
mean,1.038708e+07,5.803668e-16,-8.914035e-17,-2.847498e-16,3.292104,4676.283285,4428.688842,1921.968250,3.292100,-4.144218e-16
std,2.234803e+05,1.000001e+00,1.000001e+00,1.000001e+00,1.867014,3409.384253,3334.667387,1004.082949,1.867013,1.000001e+00
min,1.000000e+07,-1.416222e+00,-1.178170e+00,-1.224968e+00,1.000000,500.000000,500.000000,500.000000,1.000000,-1.698544e+00
25%,1.019354e+07,-7.055553e-01,-8.422286e-01,-8.471777e-01,2.000000,1787.940000,1620.190000,1213.525000,2.000000,-8.363423e-01
50%,1.038708e+07,-2.058131e-01,-3.214686e-01,-2.781284e-01,3.000000,3727.760000,3356.510000,1715.280000,3.000000,-6.716999e-02
75%,1.058062e+07,4.150377e-01,6.725282e-01,7.290283e-01,5.000000,7162.200000,6671.610000,2338.700000,5.000000,7.053880e-01
max,1.077416e+07,5.057162e+00,2.869700e+00,2.734583e+00,6.000000,13999.300000,13997.900000,6999.650000,6.000000,2.596554e+00


In [69]:
train_df.columns

Index(['Customer_ID', 'luxury_sales', 'fresh_sales', 'dry_sales',
       'cluster_catgeory', 'dry_sales_numeric', 'fresh_sales_numeric',
       'luxury_sales_numeric', 'clusterCategory_numeric',
       'outlet_city_Batticaloa', 'outlet_city_Colombo',
       'outlet_city_Dehiwala-Mount Lavinia', 'outlet_city_Galle',
       'outlet_city_Gampaha', 'outlet_city_Homagama', 'outlet_city_Jaffna',
       'outlet_city_Kaduwela', 'outlet_city_Kalmunai', 'outlet_city_Kandy',
       'outlet_city_Katunayake', 'outlet_city_Kelaniya', 'outlet_city_Kotte',
       'outlet_city_Moratuwa', 'outlet_city_Negombo',
       'outlet_city_Nuwara Eliya', 'outlet_city_Panadura',
       'outlet_city_Peliyagoda', 'outlet_city_Trincomalee',
       'outlet_city_Wattala', 'total_sales'],
      dtype='object')

In [70]:
# Select columns to include in the cleaned DataFrame
selected_columns = ['Customer_ID', 'luxury_sales', 'fresh_sales', 'dry_sales',
       'cluster_catgeory', 
       'outlet_city_Batticaloa', 'outlet_city_Colombo',
       'outlet_city_Dehiwala-Mount Lavinia', 'outlet_city_Galle',
       'outlet_city_Gampaha', 'outlet_city_Homagama', 'outlet_city_Jaffna',
       'outlet_city_Kaduwela', 'outlet_city_Kalmunai', 'outlet_city_Kandy',
       'outlet_city_Katunayake', 'outlet_city_Kelaniya', 'outlet_city_Kotte',
       'outlet_city_Moratuwa', 'outlet_city_Negombo',
       'outlet_city_Nuwara Eliya', 'outlet_city_Panadura',
       'outlet_city_Peliyagoda', 'outlet_city_Trincomalee',
       'outlet_city_Wattala', 'total_sales']
# Create the cleaned DataFrame
cleaned_train_df = train_df.loc[:, selected_columns].copy()


In [71]:
cleaned_train_df.head()

,Customer_ID,luxury_sales,fresh_sales,dry_sales,cluster_catgeory,outlet_city_Batticaloa,outlet_city_Colombo,outlet_city_Dehiwala-Mount Lavinia,outlet_city_Galle,outlet_city_Gampaha,...,outlet_city_Kelaniya,outlet_city_Kotte,outlet_city_Moratuwa,outlet_city_Negombo,outlet_city_Nuwara Eliya,outlet_city_Panadura,outlet_city_Peliyagoda,outlet_city_Trincomalee,outlet_city_Wattala,total_sales
0,10493832,-0.709489,-1.101399,0.180599,4,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,-0.982408
1,10178643,-0.330507,-1.010208,0.390355,1,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,-0.617597
2,10513916,0.365429,-0.883967,1.313829,4,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,0.494808
3,10334589,0.351545,-0.806410,1.297473,4,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,0.544047
4,10458365,0.421810,-0.707480,1.339778,4,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,0.686005


In [72]:
test_df.columns

Index(['Customer_ID', 'luxury_sales', 'fresh_sales', 'dry_sales',
       'luxury_sales_numeric', 'fresh_sales_numeric', 'dry_sales_numeric',
       'total_sales', 'outlet_city_Anuradhapura', 'outlet_city_Batticaloa',
       'outlet_city_Colombo', 'outlet_city_Dehiwala-Mount Lavinia',
       'outlet_city_Galle', 'outlet_city_Gampaha', 'outlet_city_Homagama',
       'outlet_city_Jaffna', 'outlet_city_Kaduwela', 'outlet_city_Kalmunai',
       'outlet_city_Kandy', 'outlet_city_Katunayake', 'outlet_city_Kelaniya',
       'outlet_city_Kotte', 'outlet_city_Madawachiya', 'outlet_city_MoraTuwa',
       'outlet_city_Moratuwa', 'outlet_city_Negombo',
       'outlet_city_Nuwara Eliya', 'outlet_city_Panadura',
       'outlet_city_PeliyagodA', 'outlet_city_Peliyagoda',
       'outlet_city_Trincomale', 'outlet_city_Trincomalee',
       'outlet_city_Wattala', 'outlet_city_batticaloa',
       'outlet_city_kalmunai'],
      dtype='object')

In [73]:
X_test = test_df[['luxury_sales', 'fresh_sales', 'dry_sales', 
       'outlet_city_Batticaloa', 'outlet_city_Colombo',
       'outlet_city_Dehiwala-Mount Lavinia', 'outlet_city_Galle',
       'outlet_city_Gampaha', 'outlet_city_Homagama', 'outlet_city_Jaffna',
       'outlet_city_Kaduwela', 'outlet_city_Kalmunai', 'outlet_city_Kandy',
       'outlet_city_Katunayake', 'outlet_city_Kelaniya', 'outlet_city_Kotte',
       'outlet_city_Moratuwa', 'outlet_city_Negombo',
       'outlet_city_Nuwara Eliya', 'outlet_city_Panadura',
       'outlet_city_Peliyagoda', 'outlet_city_Trincomalee',
       'outlet_city_Wattala','total_sales']]


In [74]:
# y_pred = clf.predict(X_test)

In [75]:

cleaned_train_df.to_csv('cleanedTrain.csv',index=False)